In [1]:
import os
import math

import numpy as np
import random

from tqdm import tqdm
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import torch

from sklearn.metrics import confusion_matrix, average_precision_score, roc_curve, roc_auc_score, accuracy_score,f1_score, normalized_mutual_info_score

from model.models import LBG
from model.temporalNeighbor import temporalNeighbor
from logger import logger
from collections import Counter
from sklearn.preprocessing import StandardScaler
#聚类指标：轮廓系数
from sklearn.metrics import silhouette_score

### Utility function and class
class EarlyStopMonitor(object):
    def __init__(self, max_round=10, higher_better=True, tolerance=1e-4):
        self.max_round = max_round
        self.num_round = 0

        self.epoch_count = 0
        self.best_epoch = 0

        self.last_best = None
        self.higher_better = higher_better
        self.tolerance = tolerance

    def early_stop_check(self, curr_val):

        if not self.higher_better:
            curr_val *= -1
        if self.last_best is None:
            self.last_best = curr_val
        elif (curr_val - self.last_best) / np.abs(self.last_best) < -self.tolerance:
            self.last_best = curr_val
            self.num_round = 0
            self.best_epoch = self.epoch_count
        else:
            self.num_round += 1
        self.epoch_count += 1
        print('last_best,best_epoch',self.last_best,self.best_epoch)
        return self.num_round >= self.max_round

class RandEdgeSampler(object):
    def __init__(self, src_list, dst_list):
        self.src_list = np.unique(src_list)
        self.dst_list = np.unique(dst_list)

    def sample(self, size):
        src_index = np.random.randint(0, len(self.src_list), size)
        dst_index = np.random.randint(0, len(self.dst_list), size)
        return self.src_list[src_index], self.dst_list[dst_index]

#5120,0.03达到0.977，0.039
#8000，0.03 达到0.979，0.039
#8000，0.01. nei 16达（0.9776，0.0367），（0.88，0.028）
#8000,2,8,0.01达0.9936，0.036
#8000,1,24,0.005达0.98，0.033
#8000,1,24,0.005达0.976，0.032
#8000,1,24,16head,0.005达0.974，0.0315,
#8000,1,24,,0.001加属性达0.986，0.034,
SEED = 1
ADGAT_EPOCHS = 150
BATCH_SIZE = 2000
NUM_LAYER = 1
UNIFORM = True
GPU = 0
NUM_NEIGHBORS = 200
NUM_HEADS = 16
DROP_OUT = 0.1
input_feat = 128
output_feat = 128
hidden = 128
USE_TIME = 'time'
AGG_METHOD = 'attn'
ATTN_MODE = 'prod'
objective = 'soft-boundary' #'hard'
SEQ_LEN = NUM_NEIGHBORS
LEARNING_RATE = 1e-4
DATA = 'auth'
c = None
R=0
RR = 0#阈值，判断当前点在球体外
att_feat = 4
warm_up_n_epochs = 5
nu=1e-5


logger.info('\n---------------------------------------new start training--------------------------------------\n')
logger.info('parameter,ADGAT_EPOCHS:{},BATCH_SIZE:{},NUM_LAYER:{}'.format(ADGAT_EPOCHS,BATCH_SIZE,NUM_LAYER))
logger.info('parameter,NUM_NEIGHBORS:{},NUM_HEADS:{},DROP_OUT:{},input_feat:{}'.format(NUM_NEIGHBORS,NUM_HEADS,DROP_OUT,input_feat))
logger.info('nu:{},att_feat:{},warm_up_n_epochs:{}'.format(nu,att_feat,warm_up_n_epochs))

ALLEDGEPATH= './auth/opt_TUDCC3.txt'#41590401,4603678
ADGATMODEL_SAVE_PATH = f'./save_models/ADGAT/{AGG_METHOD}-{ATTN_MODE}-{DATA}.pth'
ADGATget_checkpoint_path = lambda epoch: f'./save_checkpoints/ADGAT/{AGG_METHOD}-{ATTN_MODE}-{DATA}-{epoch}.pth'
red_team_name = './auth/nredteam.txt'
auth_att_node = "./auth/auth_att_node1.txt"
precess_file_path = './auth/new_events.txt'
logger.info('current file:{}'.format(ALLEDGEPATH))

def setup_seed(seed):
    if torch.cuda.is_available():
        logger.info("gpu cuda is available!")
        torch.cuda.manual_seed_all(seed)
    else:
        logger.info("cuda is not available! cpu is available!")
        torch.manual_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)


setup_seed(SEED)
red_team_file = set(open(red_team_name, 'r').readlines())


def model_roc_curve(test_node,FPR, TPR):
    fig_roc = plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    A, = plt.step(FPR, TPR, color="#ff5722", linestyle='-.', alpha=1, where='post', label="roc")
    font1 = {'family': 'Times New Roman',
             'weight': 'normal',
             'size': 16,
             }
    font2 = {'family': 'Times New Roman',
             'weight': 'normal',
             'size': 13,
             }
    plt.tick_params(labelsize=12)
    plt.xlabel('False positive rate', font1)
    plt.ylabel('True positive rate', font1)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])

    legend = plt.legend(handles=[A], prop=font2)

    plt.title('ROC curve', fontsize=14)

    fig_roc.savefig(test_node+".png", dpi=500, bbox_inches='tight')


def load_data(file):
    edge_train = []
    label_src = []
    label_dst = []
    node_id = set()
    train_src_l = []
    train_dst_l = []
    train_ts_l = []
    train_label_l = []
    i = 0
    with open(file, 'r', encoding='utf-8') as fi:
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            if (int(l[0]) > 150884):#72-83
                continue
            str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
            if str_line in red_team_file:
                train_label_l.append(1)
            else:
                train_label_l.append(0)
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            node_id.add(C1)
            node_id.add(C2)
            i = i + 1

            edge_train.append([C1, C2, int(l[0])])
            train_src_l.append(C1)
            label_src.append(0)
            train_dst_l.append(C2)
            label_dst.append(0)
            train_ts_l.append(int(l[0]))

    return node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst


node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst = load_data(ALLEDGEPATH)

feat_edge = np.zeros((len(train_src_l), input_feat))
max_idx = max(30000, max(node_id))  #max(node_id)27193
#max_idx = max(node_id)+1

feat_node = np.random.random((max_idx, input_feat))
feat_nodes = feat_node
# feat_node_nor = (feat_nodes - feat_nodes.min(axis=1)) / (feat_nodes.max(axis=1) - feat_nodes.min(axis=1))
Mm = StandardScaler()
feat_node_nor = Mm.fit_transform(feat_nodes)
# feat_node_nor = ((feat_nodes.T - feat_nodes.min(axis=1)) / (feat_nodes.max(axis=1) - feat_nodes.min(axis=1))).T
#feat_node_nor = (feat_nodes-feat_nodes.mean(axis=0))/feat_nodes.std(axis=0)

att_node = np.zeros((max_idx, att_feat))
with open(auth_att_node, 'r', encoding='utf-8') as fi:
     while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        att = list(map(float, l[1].split()))
        #feat_node[int(l[0].strip('C'))] = att[0:input_feat]
        att_node[int(l[0].strip('C'))] = att[-4:] #np.append(att[:8],att[-2:])
fi.close()
att_node_nor = (att_node - att_node.min(axis=0)) / (att_node.max(axis=0) - att_node.min(axis=0))
#assert(len(node_id)==len(feat_node))
assert (len(train_src_l) == len(train_dst_l))
assert (len(train_dst_l) == len(train_ts_l))
#assert(len(train_ts_l)==len(feat_edge))
train_e_idx_l = list(np.arange(len(train_src_l)))
logger.info("Finish Loading Data")

adj_list = [[] for _ in range(max_idx + 1)]
train_src_l = np.array(train_src_l)
train_dst_l = np.array(train_dst_l)
train_ts_l = np.array(train_ts_l)
train_label_l = np.array(train_label_l)
label_src = np.array(label_src)
label_dst = np.array(label_dst)
### Initialize the data structure for graph and edge sampling
for src, dst, eidx, ts in zip(train_src_l, train_dst_l, train_e_idx_l, train_ts_l):
    adj_list[src].append((dst, eidx, ts))
    #adj_list[dst].append((src, eidx, ts))
train_ngh_finder = temporalNeighbor(adj_list, uniform=UNIFORM)
### Model initialize
device = torch.device('cuda:{}'.format(GPU))
#device = torch.device('cpu')
lbg = LBG(train_ngh_finder, feat_node_nor, att_node_nor,num_layers=NUM_LAYER, use_time=USE_TIME,
            agg_method=AGG_METHOD, attn_mode=ATTN_MODE, seq_len=SEQ_LEN, n_head=NUM_HEADS,
            drop_out=DROP_OUT, node_dim=hidden, time_dim=hidden)
optimizer = torch.optim.Adam(lbg.parameters(), lr=LEARNING_RATE)
#optimizer = torch.optim.SGD(lbg.parameters(), lr=LEARNING_RATE,weight_decay=1e-2)
lbg = lbg.to(device)

num_instance = len(train_src_l)
num_batch = math.ceil(num_instance / BATCH_SIZE)
logger.info('num of training instances: {}'.format(num_instance))
logger.info('num of batches per epoch: {}'.format(num_batch))
logger.info('num of red team: {}'.format(sum(train_label_l)))

train_rand_sampler = RandEdgeSampler(train_src_l, train_dst_l)

idc = np.arange(num_instance)


def Deep_SVDD(output, R, c, nu):
    dist = torch.sum((output-c) ** 2, dim=1)
    # if objective == 'soft-boundary':
    #     scores = dist - R ** 2
    #     loss = R ** 2 + (1 / nu) * torch.mean(torch.max(torch.zeros_like(scores), scores))
    # else:
    scores = dist
    loss = torch.mean(dist)
    return scores,loss

def update_center(src_l, dst_l,ts_l,lbg,num_instance,num_batch):
    # c = torch.zeros(2 * (output_feat+att_feat)).to(device)
    c = torch.zeros(2 * (output_feat)).to(device)
    eps = 0.1
    lbg = lbg.eval()
    n_samples = 0
    idx = np.arange(num_instance)
    with torch.no_grad():
        for k in tqdm(range(num_batch)):
            s_idx = k * BATCH_SIZE
            e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
            if (s_idx == e_idx):
                continue
            batch_idx = idx[s_idx:e_idx]
            src_l_cut, dst_l_cut,ts_l_cut = src_l[batch_idx], dst_l[batch_idx],ts_l[batch_idx]
            enc = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
            n_samples += enc.shape[0]
            c += torch.sum(enc, dim=0)
        c /= n_samples
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
    return c
center = update_center( train_src_l, train_dst_l,train_ts_l,lbg,num_instance,num_batch).to(device)
ADGATearly_stopper = EarlyStopMonitor()


def get_radius(dist: torch.Tensor, nu: float):
    """Optimally solve for radius R via the (1-nu)-quantile of distances."""
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)

j=1
for epoch in (range(ADGAT_EPOCHS)):
    # Training
    # training use only training graph
    lbg.ngh_finder = train_ngh_finder
    acc, ap, f1, auc, m_loss, fpr, tpr, nmi,temR = [], [], [], [], [], [], [], [],[]
    pred_ll, true_ll, link_embed = [], [], []
    logger.info('start {} epoch'.format(epoch))
    idx = np.arange(num_instance)
    np.random.shuffle(idx)
    for k in tqdm(range(num_batch)):
        s_idx = k * BATCH_SIZE
        e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
        if (s_idx == e_idx):
            continue
        batch_idx = idx[s_idx:e_idx]
        src_l_cut, dst_l_cut, ts_l_cut= train_src_l[batch_idx], train_dst_l[batch_idx],train_ts_l[batch_idx]
        label_l_cut = train_label_l[batch_idx]
        size = len(src_l_cut)

        with torch.no_grad():
            pos_label = torch.from_numpy(label_l_cut).float().to(device)
        optimizer.zero_grad()

        lbg = lbg.train()
        # score, loss, RR = lbg(src_l_cut, dst_l_cut, ts_l_cut, center,epoch, NUM_NEIGHBORS)
        link = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
        # if (epoch > warm_up_n_epochs and objective == 'soft-boundary'):
        #     R= torch.tensor(get_radius(torch.sum((link - center) ** 2, dim=1), nu)).to(device)
        score, loss = Deep_SVDD(link, R, center, nu)
        loss.backward()
        optimizer.step()
        link_embed.extend(link.cpu())
        m_loss.append(loss.item())
        with torch.no_grad():
            lbg = lbg.eval()
            pred_score = score.cpu().detach().numpy()
            true_label = pos_label.cpu().numpy()
            pred_ll.extend(list(pred_score))
            true_ll.extend(list(true_label))
    pred_ll = np.array(pred_ll)
    true_ll = np.array(true_ll)
    link_embed = torch.stack(link_embed)
#     auc = roc_auc_score(true_ll, pred_ll)
    RR = get_radius(torch.sum((link_embed - center.cpu()) ** 2, dim=1), nu)
    tn, fp, fn, tp = confusion_matrix(true_ll, pred_ll>RR, labels=[False, True]).ravel()
    FPR = fp / (fp + tn)
    TPR = tp / (tp + fn)
    logger.info('Epoch mean loss: {}'.format(np.mean(m_loss)))
    logger.info('warm_up_n_epochs:{}'.format(warm_up_n_epochs))
    logger.info("Confusion Matrix:{}".format(confusion_matrix(true_ll, pred_ll > RR, labels=[False, True])))
    logger.info("FPR: {}".format(FPR))
    logger.info("TPR: {}".format(TPR) )
    logger.info('test RR: {}'.format(RR))
#     logger.info('train auc: {}'.format(auc))
    # logger.info('train f1: {}'.format(f1))
    if ADGATearly_stopper.early_stop_check(np.mean(m_loss)):
        logger.info('Loss is no decline over {} epochs , stop training'.format(ADGATearly_stopper.max_round))
        logger.info(f'Loading the best model at epoch {ADGATearly_stopper.best_epoch}')
        best_model_path = ADGATget_checkpoint_path(ADGATearly_stopper.best_epoch)
        lbg.load_state_dict(torch.load(best_model_path))
        logger.info(f'Loaded the best model at epoch {ADGATearly_stopper.best_epoch} for inference')
        break
    else:
        torch.save(lbg.state_dict(), ADGATget_checkpoint_path(epoch))
logger.info('Traing endding')




2023-03-11 00:27:19,162 19860 1479355589.py[line:98] - INFO: 
---------------------------------------new start training--------------------------------------

2023-03-11 00:27:19,168 19860 1479355589.py[line:99] - INFO: parameter,ADGAT_EPOCHS:150,BATCH_SIZE:2000,NUM_LAYER:1
2023-03-11 00:27:19,169 19860 1479355589.py[line:100] - INFO: parameter,NUM_NEIGHBORS:200,NUM_HEADS:16,DROP_OUT:0.1,input_feat:128
2023-03-11 00:27:19,171 19860 1479355589.py[line:101] - INFO: nu:1e-05,att_feat:4,warm_up_n_epochs:5
2023-03-11 00:27:19,172 19860 1479355589.py[line:109] - INFO: current file:./auth/opt_TUDCC3.txt
2023-03-11 00:27:19,256 19860 1479355589.py[line:113] - INFO: gpu cuda is available!
2023-03-11 00:27:26,580 19860 1479355589.py[line:224] - INFO: Finish Loading Data
2023-03-11 00:27:27,018 19860 1479355589.py[line:250] - INFO: num of training instances: 72717
2023-03-11 00:27:27,019 19860 1479355589.py[line:251] - INFO: num of batches per epoch: 37
2023-03-11 00:27:27,026 19860 1479355589.py

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:10<00:00,  3.51it/s]

2023-03-11 00:27:37,581 19860 1479355589.py[line:306] - INFO: start 0 epoch



100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:08<00:00,  4.11it/s]


2023-03-11 00:27:46,805 19860 1479355589.py[line:347] - INFO: Epoch mean loss: 97.83767329035578
2023-03-11 00:27:46,807 19860 1479355589.py[line:348] - INFO: warm_up_n_epochs:5
2023-03-11 00:27:46,819 19860 1479355589.py[line:349] - INFO: Confusion Matrix:[[    0 72716]
 [    0     0]]
2023-03-11 00:27:46,826 19860 1479355589.py[line:350] - INFO: FPR: 1.0
2023-03-11 00:27:46,831 19860 1479355589.py[line:351] - INFO: TPR: nan
2023-03-11 00:27:46,836 19860 1479355589.py[line:352] - INFO: test RR: 16.419602322961122
last_best,best_epoch 97.83767329035578 0
2023-03-11 00:27:47,002 19860 1479355589.py[line:306] - INFO: start 1 epoch


...

C:\Users\hxr\AppData\Local\Temp\2\ipykernel_20156\1479355589.py:346: RuntimeWarning: invalid value encountered in longlong_scalars
  TPR = tp / (tp + fn)
100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:08<00:00,  4.52it/s]


2023-03-11 00:49:38,948 19860 1479355589.py[line:347] - INFO: Epoch mean loss: 1.0181875921584465
2023-03-11 00:49:38,949 19860 1479355589.py[line:348] - INFO: warm_up_n_epochs:5
2023-03-11 00:49:38,956 19860 1479355589.py[line:349] - INFO: Confusion Matrix:[[70951  1765]
 [    0     0]]
2023-03-11 00:49:38,958 19860 1479355589.py[line:350] - INFO: FPR: 0.024272512239397106
2023-03-11 00:49:38,959 19860 1479355589.py[line:351] - INFO: TPR: nan
2023-03-11 00:49:38,960 19860 1479355589.py[line:352] - INFO: test RR: 6.966587092426886
last_best,best_epoch 1.0181875921584465 148
2023-03-11 00:49:39,102 19860 1479355589.py[line:306] - INFO: start 149 epoch


C:\Users\hxr\AppData\Local\Temp\2\ipykernel_20156\1479355589.py:346: RuntimeWarning: invalid value encountered in longlong_scalars
  TPR = tp / (tp + fn)
100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:08<00:00,  4.52it/s]


2023-03-11 00:49:47,576 19860 1479355589.py[line:347] - INFO: Epoch mean loss: 1.0042774290651888
2023-03-11 00:49:47,578 19860 1479355589.py[line:348] - INFO: warm_up_n_epochs:5
2023-03-11 00:49:47,586 19860 1479355589.py[line:349] - INFO: Confusion Matrix:[[70675  2041]
 [    0     0]]
2023-03-11 00:49:47,588 19860 1479355589.py[line:350] - INFO: FPR: 0.028068100555586117
2023-03-11 00:49:47,589 19860 1479355589.py[line:351] - INFO: TPR: nan
2023-03-11 00:49:47,590 19860 1479355589.py[line:352] - INFO: test RR: 6.26348388033042
last_best,best_epoch 1.0042774290651888 149
2023-03-11 00:49:47,678 19860 1479355589.py[line:364] - INFO: Traing endding


C:\Users\hxr\AppData\Local\Temp\2\ipykernel_20156\1479355589.py:346: RuntimeWarning: invalid value encountered in longlong_scalars
  TPR = tp / (tp + fn)


In [2]:
def test(test_node,lbg,R,num_instance,src_l,dst_l,ts_l,label_l,center):
    logger.info("{}".format(test_node))
    acc, ap, f1, auc, m_loss, fpr, tpr, nmi = [], [], [], [], [], [], [], []
    link_embed= []
    pred_ll, true_ll = [], []
    #np.random.shuffle(idc)
    with torch.no_grad():
        lbg = lbg.eval()
        BATCH_SIZE = 4000
        num_batch = math.ceil(num_instance / BATCH_SIZE)
        for k in tqdm(range(num_batch)):
            s_idx = k * BATCH_SIZE
            e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
            if (s_idx == e_idx):
                continue
            batch_idx = idc[s_idx:e_idx]
            src_l_cut, dst_l_cut,ts_l_cut = src_l[batch_idx], dst_l[batch_idx],ts_l[batch_idx]

            label_l_cut = label_l[batch_idx]

            pos_label = torch.from_numpy(label_l_cut).float().to(device)
            # score, _ , RR= lbg(src_l_cut, dst_l_cut, ts_l_cut, center, best_epoch, NUM_NEIGHBORS)
            link = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
            score,_= Deep_SVDD(link, R, center, nu)
            pred_score = score.cpu().detach().numpy()
            true_label = pos_label.cpu().numpy()
            pred_ll.extend(list(pred_score))
            true_ll.extend(list(true_label))
            link_embed.extend(link.cpu())
            m_loss.append(_.item())

        pred_ll = np.array(pred_ll)
        true_ll = np.array(true_ll)
        link_embed = torch.stack(link_embed)
        #acc = ((pred_ll > RR) == true_ll).mean()
        #ap = average_precision_score(true_ll, pred_ll)
        # f1 = f1_score(true_ll, pred_ll > RR)
        auc = roc_auc_score(true_ll, pred_ll)
        # RR = get_radius(torch.sum((link_embed - center.cpu()) ** 2, dim=1), nu)
        FPR_curve, TPR_curve, threshold1s = roc_curve(true_ll, pred_ll)
        logger.info("FPR_curve,: {},TPR_curve,:{}".format(FPR_curve, TPR_curve))
        # NMI = normalized_mutual_info_score(true_ll, pred_ll, average_method='arithmetic')
        tn, fp, fn, tp = confusion_matrix(true_ll, pred_ll>RR, labels=[False, True]).ravel()
        logger.info('Epoch mean loss: {}'.format(np.mean(m_loss)))
        logger.info("Confusion Matrix:{}".format(confusion_matrix(true_ll, pred_ll>RR, labels=[False, True])))
        FPR = fp / (fp + tn)
        TPR = tp / (tp + fn)
        logger.info("FPR: {}".format(FPR))
        logger.info("TPR: {}".format(TPR))
        print(FPR,TPR,auc)
        #logger.info('train acc: {}'.format(acc))
        logger.info('test RR: {}'.format(RR))
        logger.info('test auc: {}'.format(auc))
        #logger.info('train ap: {}'.format(ap))
        # logger.info('test f1: {}'.format(f1))
        # logger.info('test NMI: {}'.format(NMI))  #越高越好
        #logger.info(FPR, TPR)
        model_roc_curve(test_node,FPR_curve, TPR_curve)
        pred_label = pred_ll>RR
        redlink_idx = np.where(pred_label==1)[0]
        red_src_detect, red_dst_detect,red_tsl_detect = src_l[redlink_idx], dst_l[redlink_idx],ts_l[redlink_idx]
    return red_src_detect, red_dst_detect,red_tsl_detect,link_embed,true_ll

In [3]:
def load_data(file):
    edge_train = []
    label_src = []
    label_dst = []
    node_id = set()
    train_src_l = []
    train_dst_l = []
    train_ts_l = []
    train_label_l = []
    i = 0
    with open(file, 'r', encoding='utf-8') as fi:
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            if (int(l[0]) < 150885 ):#72-83
                 continue
            str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
            if str_line in red_team_file:
                train_label_l.append(1)
            else:
                train_label_l.append(0)
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            node_id.add(C1)
            node_id.add(C2)
            i = i + 1

            edge_train.append([C1, C2, int(l[0])])
            train_src_l.append(C1)
            label_src.append(0)
            train_dst_l.append(C2)
            label_dst.append(0)
            train_ts_l.append(int(l[0]))

    return node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst


node_id, te_src_l, te_dst_l, te_ts_l, te_label_l, label_src, label_dst = load_data(ALLEDGEPATH)
te_src_l = np.array(te_src_l)
te_dst_l = np.array(te_dst_l)
te_ts_l = np.array(te_ts_l)
te_label_l = np.array(te_label_l)
num_instance = len(te_src_l)
full_adj_list = adj_list.copy()
e_idx_l = list(np.arange(len(train_src_l)+len(te_src_l)))
for src, dst, eidx, ts in zip(te_src_l, te_dst_l, e_idx_l, te_ts_l):
    full_adj_list[src].append((dst, eidx, ts))
full_ngh_finder = temporalNeighbor(full_adj_list, uniform=UNIFORM)
lbg.ngh_finder = full_ngh_finder
idc = np.arange(num_instance)
num_batch = math.ceil(num_instance / BATCH_SIZE)
red_links = test('test for old nodes',lbg,R,num_instance,te_src_l,te_dst_l,te_ts_l,te_label_l,center)
rednodes_predict = set(red_links[0])|set(red_links[1])

2023-03-11 00:50:25,993 19860 3223813446.py[line:2] - INFO: test for old nodes


100%|██████████████████████████████████████████████████████████████████████████████| 1073/1073 [09:39<00:00,  1.85it/s]


2023-03-11 01:00:16,984 19860 3223813446.py[line:41] - INFO: FPR_curve,: [0.00000000e+00 2.33069212e-07 4.03209736e-05 ... 9.99912832e-01
 9.99913298e-01 1.00000000e+00],TPR_curve,:[0. 0. 0. ... 1. 1. 1.]
2023-03-11 01:00:17,732 19860 3223813446.py[line:44] - INFO: Epoch mean loss: 3.5881765502585723
2023-03-11 01:00:18,563 19860 3223813446.py[line:45] - INFO: Confusion Matrix:[[3555240  735331]
 [      9     633]]
2023-03-11 01:00:18,565 19860 3223813446.py[line:48] - INFO: FPR: 0.17138301638639705
2023-03-11 01:00:18,566 19860 3223813446.py[line:49] - INFO: TPR: 0.985981308411215
0.17138301638639705 0.985981308411215 0.9821348759096793
2023-03-11 01:00:18,568 19860 3223813446.py[line:52] - INFO: test RR: 6.26348388033042
2023-03-11 01:00:18,568 19860 3223813446.py[line:53] - INFO: test auc: 0.9821348759096793


In [11]:
nodes_count = dict(Counter(list(red_links[0])+list(red_links[1])))
#red_samenodes = [key for key,value in nodes_count.items()if value > 1]
node_sort=sorted(nodes_count.items(), key=lambda x: x[1], reverse=True)
red_samenodes = dict(node_sort[:80]).keys()
logger.info('num of predict red nodes: {}'.format(len(red_samenodes)))
nred_links = [(x,y) for x,y in zip(red_links[0],red_links[1])]
logger.info('num of predict red edges: {}'.format(len(nred_links)))
min_t = 0
max_t = 1296000

def inductive_link(red_samenodes,min_t,max_t):
    node_id = set()
    test_src_l = []
    test_dst_l = []
    test_ts_l = []
    test_label_l = []
    i = 0
    with open(precess_file_path, 'r', encoding='utf-8') as fi:
        fi.readline()
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            if (int(l[0]) < min_t or int(l[0]) > max_t):
                continue
            if (C1 in red_samenodes or C2 in red_samenodes):
                test_src_l.append(C1)
                test_dst_l.append(C2)
                test_ts_l.append(int(l[0]))
                node_id.add(C1)
                node_id.add(C2)
                str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
                if str_line in red_team_file:
                    test_label_l.append(1)
                else:
                    test_label_l.append(0)
            i = i + 1

    return node_id, test_src_l, test_dst_l, test_ts_l, test_label_l

test_node_id, test_src_l, test_dst_l, test_ts_l, test_label_l = inductive_link(red_samenodes,min_t,max_t)
logger.info('num of new nodes: {}'.format(len(set(test_node_id)-set(node_id))))
logger.info('num of new events: {}'.format(len(test_src_l)))
logger.info('num of red team: {}'.format(sum(test_label_l)))
test_src_l = np.array(test_src_l)
test_dst_l = np.array(test_dst_l)
test_ts_l = np.array(test_ts_l)
test_label_l = np.array(test_label_l)
num_instance = len(test_src_l)
full_adj_list = adj_list.copy()
e_idx_l = list(np.arange(len(train_src_l)+len(test_src_l)))
for src, dst, eidx, ts in zip(test_src_l, test_dst_l, e_idx_l, test_ts_l):
    full_adj_list[src].append((dst, eidx, ts))
full_ngh_finder = temporalNeighbor(full_adj_list, uniform=UNIFORM)
lbg.ngh_finder = full_ngh_finder
idc = np.arange(num_instance)
num_batch = math.ceil(num_instance / BATCH_SIZE)

2023-03-11 01:19:24,965 19860 2768182876.py[line:5] - INFO: num of predict red nodes: 80
2023-03-11 01:19:24,998 19860 2768182876.py[line:7] - INFO: num of predict red edges: 163184
2023-03-11 01:20:32,441 19860 2768182876.py[line:45] - INFO: num of new nodes: 5812
2023-03-11 01:20:32,443 19860 2768182876.py[line:46] - INFO: num of new events: 1995000
2023-03-11 01:20:32,471 19860 2768182876.py[line:47] - INFO: num of red team: 40


In [12]:
RR = 20
new_center = update_center( test_src_l, test_dst_l,test_ts_l,lbg,num_instance,num_batch).to(device)
new_redlinks = test('test for new nodes', lbg, R, num_instance, test_src_l, test_dst_l, test_ts_l, test_label_l,new_center)

100%|████████████████████████████████████████████████████████████████████████████████| 998/998 [04:55<00:00,  3.38it/s]

2023-03-11 01:25:38,298 19860 3223813446.py[line:2] - INFO: test for new nodes



100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [04:43<00:00,  1.76it/s]


2023-03-11 01:30:26,460 19860 3223813446.py[line:41] - INFO: FPR_curve,: [0.00000000e+00 5.01263434e-07 2.38100131e-04 ... 9.99604503e-01
 9.99605506e-01 1.00000000e+00],TPR_curve,:[0. 0. 0. ... 1. 1. 1.]
2023-03-11 01:30:26,772 19860 3223813446.py[line:44] - INFO: Epoch mean loss: 17.017244337078086
2023-03-11 01:30:27,074 19860 3223813446.py[line:45] - INFO: Confusion Matrix:[[1463479  531480]
 [      4      36]]
2023-03-11 01:30:27,076 19860 3223813446.py[line:48] - INFO: FPR: 0.26641149016095067
2023-03-11 01:30:27,077 19860 3223813446.py[line:49] - INFO: TPR: 0.9
0.26641149016095067 0.9 0.8637177631219489
2023-03-11 01:30:27,079 19860 3223813446.py[line:52] - INFO: test RR: 20
2023-03-11 01:30:27,080 19860 3223813446.py[line:53] - INFO: test auc: 0.8637177631219489
